<a href="https://colab.research.google.com/github/pcbzmani/SQL_Learning/blob/main/SQL_Problem_Solving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install delta-spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from pyspark.sql import SparkSession
from delta import *
builder= SparkSession.builder.appName('Deltalake')\
         .config('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension')\
         .config('spark.sql.catalog.spark_catlog','org.apache.spark.sql.delta.catalog.DeltaCatalog')\
         .config('spark.jars.packages','io.delta:delta-core_2.12:2.0.0')  

In [35]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

1. **Team Winning Percentage**
**Get Team winning percentage. You have given with the data of Matches, and winning team from this we need to derive Winning Percantage of each team**

In [21]:
data = [('IND','SA','IND'),('SA','IND','SA'),('IND','PAK','IND'),('AUS','PAK','AUS'),('SL','AUS','AUS')]
df = spark.createDataFrame(data,['Team1','Team2','Winning_Team'])

In [13]:
df.createOrReplaceTempView('teamsstats')

In [14]:
df.show()

+-----+-----+------------+
|Team1|Team2|Winning_Team|
+-----+-----+------------+
|  IND|   SA|         IND|
|   SA|  IND|          SA|
|  IND|  PAK|         IND|
|  AUS|  PAK|         AUS|
|   SL|  AUS|         AUS|
+-----+-----+------------+



In [19]:
res = spark.sql("""
with 
wining_list as (
  select 
    Winning_Team,
    count(*) as wincount 
  from teamsstats 
    group by Winning_Team),
team_play_cnt as (
  select team,
          count(*) as totplay 
      from ( select 
                    Team1 as team 
              from teamsstats 
                union all 
           select 
                  Team2 as team 
           from teamsstats ) a 
           group by a.team)

select 
    team, 
    round(coalesce(wincount,0)/totplay,2)*100 as win_percent 
from  team_play_cnt
left join  wining_list  
on team_play_cnt.team = wining_list.Winning_Team
order by round(coalesce(wincount,0)/totplay,2)*100 desc""")

In [20]:
res.show()

+----+-----------+
|team|win_percent|
+----+-----------+
| AUS|      100.0|
| IND|       67.0|
|  SA|       50.0|
|  SL|        0.0|
| PAK|        0.0|
+----+-----------+



2. **Count of Passengers in Bus** 
 
 We need to idenify how many passengers boarded on a particular bus

 Expected result for below one is 
        10 -> 0
        20 -> 1
        21 -> 3
        22 -> 1
        30 -> 1




In [23]:
bus_data = [(10,'W','B','10:55'),(20,'B','P','06:20'),(21,'B','P','14:00'),(22,'B','P','21:40'),(30,'P','M','13:30')]
bus_col = ['ID','Origin','Destn','Time']
pass_data = [(1,'P','M','13:30'),(2,'P','M','13:31'),(10,'W','P','10:00'),(11,'W','B','22:31'),(40,'B','P','06:15'),(41,'B','P','06:50'),\
            (42,'B','P','07:12'),(43,'B','P','12:03'),(44,'B','P','20:00') ]
pass_col = ['id','Origin','Destn','time']

In [32]:
bus_df = spark.createDataFrame(bus_data,bus_col)
bus_df.createOrReplaceTempView('bus')
pass_df = spark.createDataFrame(pass_data,pass_col)
pass_df.createOrReplaceTempView('passenger')

In [26]:
bus_df.show()

+---+------+-----+-----+
| ID|Origin|Destn| Time|
+---+------+-----+-----+
| 10|     W|    B|10:55|
| 20|     B|    P|06:20|
| 21|     B|    P|14:00|
| 22|     B|    P|21:40|
| 30|     P|    M|13:30|
+---+------+-----+-----+



In [27]:
pass_df.show()

+---+------+-----+-----+
| id|Origin|Destn| time|
+---+------+-----+-----+
|  1|     P|    M|13:30|
|  2|     P|    M|13:31|
| 10|     W|    P|10:00|
| 11|     W|    B|22:31|
| 40|     B|    P|06:15|
| 41|     B|    P|06:50|
| 42|     B|    P|07:12|
| 43|     B|    P|12:03|
| 44|     B|    P|20:00|
+---+------+-----+-----+



In [52]:
df = spark.sql("""
with 
bus_prev_time as(
    select ID, Origin,Destn,coalesce(prevtime,'00:00') as prev_time,Time from (
    select ID, Origin,Destn, Time, lag(Time) over(partition by Origin,Destn order by Time asc) as prevtime from bus) a
),
passenger as (
    select Origin,Destn, time from passenger
)

select a.ID, sum(a.passcount) from (
select b.ID,
      case when p.time is null 
            then 0
      else 1 end as passcount 
from bus_prev_time b
left join passenger p
on b.Origin = p.Origin
and b.Destn = p.Destn
and p.time between b.prev_time and b.Time ) a 
group by a.ID
order by a.ID""")

In [53]:
df.show()

+---+--------------+
| ID|sum(passcount)|
+---+--------------+
| 10|             0|
| 20|             1|
| 21|             3|
| 22|             1|
| 30|             1|
+---+--------------+



**3. Total balance of a customer**

You are given with transaction details of payment app. You need to identify the total balance of each customer

In [6]:
transaction_data = [(1,'AA','BA',120.22),(2,'BA','CA',100.55),(3,'AA','CA',250.22),(4,'CA','AA',34.55),(5,'CA','BA',55.55),(6,'ZA','AA',450.33),\
                    (7,'DA','ZA',456.55),(8,'BA','ZA',55.78)]
trans_schema = ['trans_id','sender','receiver','amount']
trans_df = spark.createDataFrame(transaction_data,trans_schema)

In [15]:
trans_df.createOrReplaceTempView('trans')

In [7]:
trans_df.show()

+--------+------+--------+------+
|trans_id|sender|receiver|amount|
+--------+------+--------+------+
|       1|    AA|      BA|120.22|
|       2|    BA|      CA|100.55|
|       3|    AA|      CA|250.22|
|       4|    CA|      AA| 34.55|
|       5|    CA|      BA| 55.55|
|       6|    ZA|      AA|450.33|
|       7|    DA|      ZA|456.55|
|       8|    BA|      ZA| 55.78|
+--------+------+--------+------+



In [25]:
df = spark.sql("""
with 
people as 
  (select distinct a.ppl from 
      (select  sender as ppl from trans 
          union all 
       select  receiver as ppl from trans)a),
debit as 
  (select sender,
          sum(amount) as debited 
    from trans 
      group by sender ),
credit as (
      select receiver,
            sum(amount) as credited 
      from trans 
      group by receiver)

select ppl,
        round(coalesce(credited,0),2) as credit, 
        round(coalesce(debited,0),2) as debit,
        round((coalesce(credited,0)-coalesce(debited,0)),2) as totalbal from 
    people peo 
    left join debit db
        on peo.ppl = db.sender
    left join credit cr
        on peo.ppl = cr.receiver
    order by ppl
""")

In [26]:
df.show()

+---+------+------+--------+
|ppl|credit| debit|totalbal|
+---+------+------+--------+
| AA|484.88|370.44|  114.44|
| BA|175.77|156.33|   19.44|
| CA|350.77|  90.1|  260.67|
| DA|   0.0|456.55| -456.55|
| ZA|512.33|450.33|    62.0|
+---+------+------+--------+



**4.Credit card charges**

Find the credit card usage of a customer.
  1. If Number of transactions is greater than or equal to 3 and total spend is above Rs.100 or total spend is greater than Rs.150 then that month card charge free, else Rs.5 needs to pay as card charge.
  2. In the below scenario, customer needs to pay Rs.5 for 8 months. Only for 4 months he/she is eliglible for concession from above case

In [48]:
credit_card_data = [('2021-01-01',50.00),('2021-01-05',25.00),('2021-01-08',10.00),('2021-01-30',15.00),\
                    ('2021-02-06',45.30),('2021-02-10',65.00),\
                    ('2021-03-01',12.30),('2021-03-10',34.00),('2022-03-24',100.00),\
                    ('2021-04-01',18.90),('2021-04-16',23.00),('2022-04-27',90.00),\
                    ('2021-06-01',45.90),('2021-06-19',65.00),('2022-06-30',78.00),\
                    ('2021-08-01',100.90),('2021-09-19',123.00),\
                    ('2021-12-01',300.00)]
credit_schema = ['date','amount']
credit_card_df = spark.createDataFrame(credit_card_data,credit_schema)
credit_card_df.createOrReplaceTempView('creditcard')

In [49]:
credit_card_df.show()

+----------+------+
|      date|amount|
+----------+------+
|2021-01-01|  50.0|
|2021-01-05|  25.0|
|2021-01-08|  10.0|
|2021-01-30|  15.0|
|2021-02-06|  45.3|
|2021-02-10|  65.0|
|2021-03-01|  12.3|
|2021-03-10|  34.0|
|2022-03-24| 100.0|
|2021-04-01|  18.9|
|2021-04-16|  23.0|
|2022-04-27|  90.0|
|2021-06-01|  45.9|
|2021-06-19|  65.0|
|2022-06-30|  78.0|
|2021-08-01| 100.9|
|2021-09-19| 123.0|
|2021-12-01| 300.0|
+----------+------+



In [60]:
df= spark.sql("""
  select ((12 - b.cnt) * 5) as fee 
  from (
    select count(*) as cnt from (
          select 
              month(date) as mnth, 
              sum(amount) as mnth_trans,
              count(*) as total_trans 
          from creditcard
              group by  month(date)) a
    where (a.total_trans >=3 and a.mnth_trans > 100.00) 
    or a.mnth_trans >= 150.00
  ) b
""")

In [61]:
df.show()

+---+
|fee|
+---+
| 40|
+---+

